In [16]:
import sys

from nltk.util import pr
sys.path.append('../')
from pathlib import Path
import json
import os
from os import name
from scripts.preprocessor import Preprocessor
from scripts.tools import *
import nltk
from nltk import word_tokenize
import re
import spacy
nlp = spacy.load('en_core_web_sm')
import random
import pandas as pd
from spacy import displacy


In [17]:
basedir = "/home/ubuntu/random_forest/recipe_classifier"

preprocessor = Preprocessor(f'{basedir}/meta/units.json', f'{basedir}/meta/cuisines.json')



In [18]:
preprocessor.lemmatizer.lemmatize("ground")

'ground'

In [19]:
nltk.help.upenn_tagset('NNS')

NNS: noun, common, plural
    undergraduates scotches bric-a-brac products bodyguards facets coasts
    divestitures storehouses designs clubs fragrances averages
    subjectivists apprehensions muses factory-jobs ...


In [20]:
text = word_tokenize("chopped fresh flat leaf parsley")
nltk.pos_tag(text)


[('chopped', 'VBN'),
 ('fresh', 'JJ'),
 ('flat', 'JJ'),
 ('leaf', 'NN'),
 ('parsley', 'NN')]

In [21]:
def get_unit_list(path_to_units: str):
        '''
        returns list of units of measurment,stored in the units json file,in the correct order.
        '''
        with open(path_to_units, 'r') as f:
            raw = f.read()
            data = json.loads(raw)
            unit_list = data['units']
            result = []
            for unit_data in unit_list:
                # plural has to come first because of the way regex matches words.
                plural = unit_data['plural']
                unit = unit_data['unit']
                abbreviation = unit_data['abbreviation']

                if plural != '':
                    result.append(plural)
                if unit != '':
                    result.append(unit)
                if abbreviation != '':
                    result.append(abbreviation)
            return result

In [22]:
def preprocess(X):
    result = []

    surplus_ingredients = []

    units = get_unit_list(path_to_units=f'{basedir}/meta/units.json')
    unit_pattern = [fr' {unit} ' for unit in units]
    unit_pattern = f'({"|".join(unit_pattern)})'

    for x in X:

        # remove brackets and content
        _x = []
        bracket = False
        for word in x:
            if word == "(":
                bracket = True

            if not bracket:
                _x.append(word)

            if word == ")":
                bracket = False

        x = ''.join(_x)


        # remove all numbers
        x = re.sub(r'\d', '', x)

        

        # removing units
        x = re.sub(unit_pattern, '', x)

        # # remove remaining unwanted char

        # x = re.sub(r'(\s-\s|_|/)', ' ', x)

        # x = re.findall("[A-Za-z0-9-']*", x)
        # x = ' '.join(x)

        # remove spaces
        x = x.strip()

        _x = []
        for i in range(len(x)):
            if x[i] == ' ' and i < len(x)-1:
                if x[i+1] != ' ':
                    _x.append(x[i])
            else:
                _x.append(x[i])

        x = ''.join(_x)

        #handling, and, plus,...
        
        # conjuctions = ['and','plus']

        # present = []
        # _x  = x
        # for word in conjuctions:
        #     x.


        


        result.append(x)

    return result


In [75]:
def preprocess_single_ingredient(ing, display=False):
    surplus_ingredients = []    

    print(ing)

    with open(f'{basedir}/meta/colors.json','r') as f:
        raw = f.read()
        data = json.loads(raw)
        colors = data['colors']

    units = get_unit_list(path_to_units=f'{basedir}/meta/units.json')
    unit_pattern = [fr' {unit} ' for unit in units]
    unit_pattern = f'({"|".join(unit_pattern)})'

    unwanted_adjectives = []
    adjectives = []

    with open(f'{basedir}/meta/adjectives.json','r') as f:
        raw = f.read()
        data = json.loads(raw)

    adjectives = set(data['adjectives'])

    with open(f'{basedir}/meta/coocking_adjectives/coocking_adjectives.json','r') as f:
        raw = f.read()
        data = json.loads(raw)
        
        for key in data:
            unwanted_adjectives.extend(data[key])

    unwanted_adjectives = set(unwanted_adjectives)


    # remove brackets and content
    _ing = []
    bracket = False
    for word in ing:
        if word == "(":
            bracket = True

        if not bracket:
            _ing.append(word)

        if word == ")":
            bracket = False

    ing = ''.join(_ing)

    # remove all numbers
    ing = re.sub(r'\d', ' ', ing)

    # removing units
    ing = re.sub(unit_pattern, ' ', ing)

    #getting base ingredient
    ing_string = f'I ate {ing}'
    # print(f'ingstring: {ing_string}')



    #remove words after slashes 

    ing_string = re.sub('/',' / ',ing_string)

    ing_string = ing_string.split()

    _ing_string = []

    previous = None
    for w in ing_string:
        if previous != '/' and w != '/':
            _ing_string.append(w)
        
        previous = w

    ing_string = " ".join(_ing_string)



    # remove remaining unwanted char

    # ing_string = re.sub(r'(\s-\s|_|/)', ' ', ing_string)

    ing_string = re.findall("[A-Za-z0-9-']*", ing_string)
    ing_string = ' '.join(ing_string)






    # remove spaces
    ing_string = ing_string.strip()

    _ing_string = []
    for i in range(len(ing_string)):
        if ing_string[i] == ' ' and i < len(ing_string)-1:
            if ing_string[i+1] != ' ':
                _ing_string.append(ing_string[i])
        else:
            _ing_string.append(ing_string[i])

    ing_string = ''.join(_ing_string)




    text = nlp(ing_string)
    parsed_text = nlp(ing_string)
    
    if display:
        displacy.serve(parsed_text, style="dep")

    parsed_text = [i for i in parsed_text if str(i) != ' ']

    base_ing = ''

    for word in parsed_text:
        # print(word,word.dep_,word.pos_,word.tag_)

        #dobj for direct object
        if word.dep_ == "dobj":
            base_ing = word.orth_


    if base_ing == '':
        return base_ing
    
    # print(f'Base_ing: {base_ing}')
    
    text = [str(i) for i in list(parsed_text)]
    
    ind = text.index(base_ing)
    result = []
    tags = nltk.pos_tag(text)
    
    i = ind - 1
    # print(text)
    wanted = set(['JJ','JJS','NNP','NNPS'])
    word_list = list(parsed_text)
    # while i > 0 and word_list[i].tag_ in wanted:
    #     print(word_list[i].tag_)
    #     base_ing = f'{text[i]} {base_ing}' 
    #     i -= 1
    while i > 0:
        # print(word_list[i],word_list[i].tag_)

        '''
        select word if
        1.) word tag in wanted 
        2.) word is a color
        3.) word is a compound noun
        4.) word not in unwanted_adjectives
        '''
        if word_list[i].tag_ in wanted or text[i] in colors or word_list[i].dep_ == 'compound':
            # print(word_list[i].tag_)
            
            if text[i] not in unwanted_adjectives and (not(text[i] in adjectives) and not (text[i] in colors)):
                base_ing = f'{text[i]} {base_ing}'  
        i -= 1
    print(f'ing:{ing}{" "*(70-len(ing))}Base_ing: {base_ing}\n')

    return base_ing

In [24]:
def preprocess_ingredients(ingredients):
    result = []
    conjunctions = ["and",'or']

    for ing_list in ingredients:
        try:
            length = len(ing_list)
            result = []
            start = True
            _ing_list = ing_list
            while start or length != len(ing_list):
                for conj in conjunctions:
                    for ing in _ing_list:
                        ing = _ing_list.pop(0)
                        _ing_list.extend(ing.split(conj))

                length = len(_ing_list)
                
                start = False
            result.append(_ing_list)    
        except Exception as e:
            pass

    
    # result = [[preprocess_single_ingredient(ing) for ing in ing_list] for ing_list in result]

    _result = []
    for ing_list in result:
        try:
            _result.append([preprocess_single_ingredient(ing) for ing in ing_list])
        except Exception as e:
            pass

    return result


In [70]:
test = "½ suran / yam / suvarnagadde "
# test ="I ate nuts/bla"

test = re.sub('/',' / ',test)

test = test.split()


print(test)

result = []
unwanted = [' ','/']

slash = False
previous = None
for w in test:
    if previous != '/' and w != '/':
        result.append(w)
    
    previous = w

result = " ".join(result)
print(result)

['½', 'suran', '/', 'yam', '/', 'suvarnagadde']
½ suran


In [25]:
test = "Feta cheese and/or mozerella cheese and balls"
test = "Feta cheese or mozerella cheese and balls"
# print(test.split('and','or'))
ingredients=[[test]]
preprocess_ingredients(ingredients=ingredients)

Feta NNP
ate VBD
ing:Feta cheese  Base_ing: Feta cheese
mozerella NN
ate VBD
ing: mozerella cheese  Base_ing: mozerella cheese
ate VBD
ing: balls Base_ing: balls


[['Feta cheese ', ' mozerella cheese ', ' balls']]

In [26]:
test = "Feta cheese and/or mozerella cheese and nuts"
list(test.split("andd"))

['Feta cheese and/or mozerella cheese and nuts']

In [27]:
test = 'fresh feta cheese crumbles'
# test = 'feta cheese crumbles'
# test = 'romaine lettuce';
result = preprocess_single_ingredient(test)
print(f'result: {result}')

cheese NN
feta JJ
fresh JJ
ate VBD
ing:fresh feta cheese crumbles Base_ing: feta cheese crumbles
result: feta cheese crumbles


In [28]:
test = 'fresh feta cheese crumbles'
# test = 'feta cheese crumbles'
# test = 'romaine lettuce';
preprocess_single_ingredient(test)

cheese NN
feta JJ
fresh JJ
ate VBD
ing:fresh feta cheese crumbles Base_ing: feta cheese crumbles


'feta cheese crumbles'

In [76]:
test = 'soy sauce'
test = 'medium shrimp'
test = 'thai sauce'
test = 'skinless, boneless chicken breast halves'
test = '1 ½ teaspoons cayenne pepper'
test = 'packed fresh flat-leaf parsley leaves'
test = '½ red pepper flakes, to taste'
test = '¼ red wine vinegar'
test = '½ teaspoon fine sea salt (if using regular table salt, scale back a bit)'
test = 'fine-grain sea salt'
test = 'low fat mozzarella cheese'
# test = 'delicious apples'
preprocess_single_ingredient(test,True)

low fat mozzarella cheese



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
ing:low fat mozzarella cheese                                             Base_ing: cheese



'cheese'

In [ ]:
test = "½ (12 fluid ounce) can brandy-based orange liqueur (such as Grand Marnier®)"
# test = "½ avocado - peeled, pitted, and thinly sliced"
# test = "pig's trotters"
# test = "no-salt-added black beans"
# test = "½+ tablespoon chopped chipotle pepper in adobo* (start with ½ tablespoon and add more to taste, I thought mine was just right with 1 tablespoon)"
# test = "⅓ cup old-fashioned oats, plus 2 tablespoons for sprinkling on top"
# test = "⅓ cup homemade muesli, or ⅓ cup old-fashioned oats plus ¼ teaspoon ground cinnamon"
# test = "1 ounce (2 tablespoons/about ½ of a medium orange) fresh orange juice"
# test = "½ cup suran / yam / suvarnagadde (cubed)"
# test = "½ cup thinly sliced Kalamata olives and/or roughly chopped pepper rings"
# test = "⅓ cup homemade muesli, or ⅓ cup old-fashioned oats plus ¼ teaspoon ground cinnamon"
# test = "no-salt-added black beans"
# test = "½ cup dry lentils (French green lentils or regular brown lentils), or 1 ½ cups cooked lentils (leftover or from a can, rinsed and drained)"
# test = '½ small red onion, chopped (mine came to about ¾ cup chopped)'
# test = '1 pound rhubarb (about 4 stalks), cut into ¼ by ½-inch pieces (slice larger stalks in half lengthwise before slicing them into ¼-inch wide pieces)'


In [ ]:
# # Load English tokenizer, tagger, 
# # parser, NER and word vectors
# nlp = spacy.load("en_core_web_sm")
  
# # Process whole documents
# text = ("I ate cubed cheddar cheese")
# # text = ("roasted red peppers, patted dry and roughly chopped")
  
# parsed_text = nlp(text)
# print(list(parsed_text))
# for word in parsed_text:
#   if word.dep_ == "dobj":
#       direct_object = word.orth_
  
# lemmatizer = nlp.get_pipe("lemmatizer")
# # nlp.add_pipe("morphologizer")
# # Rule-based lemmatizer
# nlp.add_pipe("lemmatizer", config={"mode": "lookup"})




# for t in parsed_text:
#   print(t,t.lemma_,t.morph,t.tag_)
# # print(doc)

# # # Token and Tag
# # tmp = []
# # wanted = ['ADJ','']
# # for token in doc:
# #   print(token, token.pos_)

Problemen

½ cup salsa verde, homemade or jarred, or your favorite salsa

½ cup thinly sliced Kalamata olives and/or roughly chopped pepper rings

½ cup suran / yam / suvarnagadde (cubed)

1 serrano chile pepper, seeded and minced


In [ ]:
result = preprocess([test])
print(result)

['½ brandy-based orange liqueur']


In [ ]:
text = word_tokenize(f'I ate 1 pound rhubarb (about 4 stalks), cut into ¼ by ½-inch pieces (slice larger stalks in half lengthwise before slicing them into ¼-inch wide pieces)')
text = word_tokenize(f'I ate 1 pound rhubarb (about 4 stalks), cut into ¼ by ½-inch pieces (slice larger stalks in half lengthwise before slicing them into ¼-inch wide pieces)')
print(text)
result = nltk.pos_tag(text)
print(result)
_tmp = []
wanted = ['NN','NNS','JJ']
for word,tag in result:
    if tag in wanted:
        _tmp.append(word) 
print(' '.join(_tmp))

['I', 'ate', '1', 'pound', 'rhubarb', '(', 'about', '4', 'stalks', ')', ',', 'cut', 'into', '¼', 'by', '½-inch', 'pieces', '(', 'slice', 'larger', 'stalks', 'in', 'half', 'lengthwise', 'before', 'slicing', 'them', 'into', '¼-inch', 'wide', 'pieces', ')']
[('I', 'PRP'), ('ate', 'VBP'), ('1', 'CD'), ('pound', 'NN'), ('rhubarb', 'NN'), ('(', '('), ('about', 'IN'), ('4', 'CD'), ('stalks', 'NNS'), (')', ')'), (',', ','), ('cut', 'VBN'), ('into', 'IN'), ('¼', 'VBN'), ('by', 'IN'), ('½-inch', 'JJ'), ('pieces', 'NNS'), ('(', '('), ('slice', 'NN'), ('larger', 'JJR'), ('stalks', 'NNS'), ('in', 'IN'), ('half', 'JJ'), ('lengthwise', 'NN'), ('before', 'IN'), ('slicing', 'VBG'), ('them', 'PRP'), ('into', 'IN'), ('¼-inch', 'JJ'), ('wide', 'JJ'), ('pieces', 'NNS'), (')', ')')]
pound rhubarb stalks ½-inch pieces slice stalks half lengthwise ¼-inch wide pieces


In [ ]:
# nlp = spacy.load('en')
sent = 'thinly sliced Kalamata olives and or roughly chopped pepper rings'
doc=nlp(sent)

sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj") ]

print(sub_toks) 

[]


In [ ]:

parsed_text = nlp("½ cup thinly sliced Kalamata olives and/or roughly chopped pepper rings")

#get token dependencies
for text in parsed_text:
    #subject would be
    if text.dep_ == "nsubj":
        subject = text.orth_
    #iobj for indirect object
    if text.dep_ == "iobj":
        indirect_object = text.orth_
    #dobj for direct object
    if text.dep_ == "dobj":
        direct_object = text.orth_

print(subject)
print(direct_object)
# print(indirect_object)

cup
rings


In [30]:

INPUT_PATH = f'{basedir}/data/train.json'
# read data from dataset from kaggle
with open(INPUT_PATH, 'r') as f:
    raw = f.read()
    data = json.loads(raw)

X = [sample['ingredients'] for sample in data]
X = preprocessor.preprocess_ingredients(X)

Y = [sample['cuisine'] for sample in data]
Y = preprocessor.preprocess_cuisines(Y)


In [31]:
random.shuffle(X)

ing:pasta sauce Base_ing: pasta
ing:grated parmesan cheese Base_ing: cheese
ing:dried parsley Base_ing: parsley
ing:water Base_ing: water
ing:shredded mozzarella cheese Base_ing: cheese
ing:lasagna noodle Base_ing: lasagna
ing:sour cream Base_ing: cream
ing:cottage cheese Base_ing: cheese
ing:star anise Base_ing: star


In [32]:
preprocess_ingredients(X)

white JJ
ate VBD
ing:white wine Base_ing: wine
chicken NN
skinless NN
boneless JJ
ate VBD
ing:boneless skinless chicken breast half Base_ing: boneless skinless chicken breast
ate VBD
ing:butter Base_ing: butter
ate VBD
ing:prosciutto Base_ing: prosciutto
provolone JJ
ate VBD
ing:provolone cheese Base_ing: provolone cheese


[['white wine',
  'boneless skinless chicken breast half',
  'butter',
  'prosciutto',
  'provolone cheese']]

In [33]:
_X = [' '.join(x) for x in X]
cheeses = []
for x in _X:
    if 'cheese' in x:
        tmp = x.split()
        # print(tmp)
        if 'cheese' in tmp: 
            i = tmp.index('cheese')

            cheeses.append(tmp[i-3:i+1])

In [34]:
for c in cheeses:
    if c != []:
        print(c)

['pesto', 'part-skim', 'mozzarella', 'cheese']
['cream', 'shredded', 'cheddar', 'cheese']
['pasta', 'grated', 'parmesan', 'cheese']
['sharp', 'white', 'cheddar', 'cheese']
['mix', 'shredded', 'cheddar', 'cheese']
['black', 'pepper', 'goat', 'cheese']
['fat', 'free', 'cream', 'cheese']
['italian', 'sausage', 'provolone', 'cheese']
['oil', 'shredded', 'mozzarella', 'cheese']
['oil', 'shredded', 'mozzarella', 'cheese']
['chili', 'powder', 'muenster', 'cheese']
['salt', 'fresh', 'parmesan', 'cheese']
['basil', 'grated', 'parmesan', 'cheese']
['crumbles', 'part-skim', 'mozzarella', 'cheese']
['spinach', 'onion', 'feta', 'cheese']
['purple', 'onion', 'fontina', 'cheese']
['milk', 'cinnamon', 'cottage', 'cheese']
['juice', 'roquef', 't', 'cheese']
['shredded', 'sharp', 'cheddar', 'cheese']
['onion', 'grated', 'jack', 'cheese']
['egano', 'grated', 'parmesan', 'cheese']
['lime', 'garlic', 'reduced-fat', 'cheese']
['black', 'pepper', 'feta', 'cheese']
['sausage', 'shredded', 'cheddar', 'cheese']